In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.utils import shuffle

In [2]:
X = np.load('combined.npy')
y = np.load('segmented.npy')

In [3]:
print(X.shape, y.shape)

(2000, 64, 84) (2000, 64, 84, 10)


In [4]:
#Shuffle
X,y = shuffle(X,y)

X = X.astype(np.float32)
y = y.astype(np.float32)

In [5]:
X = (X.astype(np.float32)-127)/127.0

In [6]:
from tensorflow.core.framework import graph_pb2

In [7]:
graph_def = graph_pb2.GraphDef()

In [8]:
def load_graph(graph_path):
    with open(graph_path, "rb") as f:
        graph_def.ParseFromString(f.read())
        with tf.Graph().as_default() as graph:
            input_big = tf.placeholder(dtype=tf.float32, shape=(None, 64,84,1), name='input_big_image')
            tf.import_graph_def(graph_def, name="", input_map={"reshaped_image": input_big})
            return graph

In [9]:
nb_classes=10

In [16]:
def nn(nb_classes):
    encoder = load_graph('./checkpoints/frozen_graph.pb')
    with encoder.as_default():
        input_ph = encoder.get_tensor_by_name('input_big_image:0')
        maxpool1_out = encoder.get_tensor_by_name('pool1/MaxPool:0')
        maxpool2_out = encoder.get_tensor_by_name('poool2/MaxPool:0')
    
        conv_11 = tf.layers.conv2d(maxpool2_out,32,1,1,activation=tf.nn.relu, name='conv11')
    
        return conv_11

In [18]:
layer = nn(nb_classes)
print(layer.shape)

(?, 14, 19, 32)
